<a href="https://colab.research.google.com/github/Thambara-20/spm-planbot/blob/main/PlanBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch datasets


In [ ]:
import json
from datasets import Dataset

# Load dataset from the local Colab path
with open('/content/scrum_activities.json') as f:
    data = json.load(f)

dataset = Dataset.from_list(data)

# Convert flattened data to a Hugging Face dataset format
print(dataset)


Dataset({
    features: ['activity', 'input_text', 'output_text'],
    num_rows: 16
})


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

# Define the tokenize function for each input-output pair
def tokenize_function(example):
    prompt = example["input_text"] + tokenizer.eos_token  # Add end-of-sequence token to input
    target = example["output_text"] + tokenizer.eos_token  # Add end-of-sequence token to output

    print("prompt:", prompt)
    print("target", target)
    # Tokenize prompt and target
    input_ids = tokenizer(prompt, truncation=True, padding='max_length', max_length=128)["input_ids"]
    target_ids = tokenizer(target, truncation=True, padding='max_length', max_length=128)["input_ids"]

    # Prepare labels for training (mask padding tokens to -100)
    labels = [-100 if token == tokenizer.pad_token_id else token for token in target_ids]

    return {"input_ids": input_ids, "labels": labels}

# Apply tokenizer to the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=False)


Map:   0%|          | 0/16 [00:00<?, ? examples/s]

prompt: Help me plan tasks for the sprint focused on login features.<|endoftext|>
target To plan for login features, break down tasks like implementing the authentication API, designing the login UI, and integrating UI with the backend. Prioritize based on dependencies.<|endoftext|>
prompt: How should we prioritize tasks in sprint planning?<|endoftext|>
target Start by prioritizing tasks that provide the most value to the end user, and consider dependencies. High-impact tasks with fewer dependencies should be prioritized.<|endoftext|>
prompt: What should I share in today's stand-up?<|endoftext|>
target In your stand-up update, share what you completed yesterday, any blockers, and what you plan to work on today. Keep it brief and focused.<|endoftext|>
prompt: I'm blocked on a task due to missing permissions. What should I do?<|endoftext|>
target Mention this blocker in the stand-up so your team or manager can assist in resolving it. If urgent, reach out to the appropriate person directl

In [ ]:
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="/content/gpt2_sprint_model",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_dir='/content/logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
)

trainer.train()


Step,Training Loss


TrainOutput(global_step=80, training_loss=0.5449258804321289, metrics={'train_runtime': 81.6066, 'train_samples_per_second': 1.961, 'train_steps_per_second': 0.98, 'total_flos': 37148027781120.0, 'train_loss': 0.5449258804321289, 'epoch': 10.0})

In [ ]:
model.save_pretrained("/content/gpt2_sprint_model")
tokenizer.save_pretrained("/content/gpt2_sprint_model")



('/content/gpt2_sprint_model/tokenizer_config.json',
 '/content/gpt2_sprint_model/special_tokens_map.json',
 '/content/gpt2_sprint_model/vocab.json',
 '/content/gpt2_sprint_model/merges.txt',
 '/content/gpt2_sprint_model/added_tokens.json')

In [ ]:
from transformers import pipeline

# Load the trained model
model = GPT2LMHeadModel.from_pretrained("/content/gpt2_sprint_model")
tokenizer = GPT2Tokenizer.from_pretrained("/content/gpt2_sprint_model")

# Set up the pipeline for text generation on GPU if available
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)  # use device=0 for GPU

# Generate output for a test input
input_text = "Help me plan tasks for the sprint focused on login features."
output = generator(input_text, max_length=100, num_return_sequences=1, truncation=True)
print(output[0]['generated_text'])


Help me plan tasks for the sprint focused on login features. What should I focus on for the sprint? What should I share in today's sprint retrospective?

During the sprint planning stage, we prioritize the tasks that we should take on based on our sprint goals. How should we prioritize tasks during refinement? What should we share in today's sprint retrospective?

Once we've identified the areas we should prioritize during refinement, we share that information with the sprint team. What should we discuss during
